In [1]:
from pyspark.ml.clustering import LDA
from pyspark.mllib.util import MLUtils

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [3]:
spark = SparkSession.builder.appName('Project').getOrCreate()

dataset=spark.read.csv("reviews.tbl", inferSchema = True, header = True, sep = '|')

dataset.createTempView("product_reviews")

In [4]:
q="SELECT CASE pr_rating WHEN 1 THEN 'NEG' WHEN 2 THEN 'NEG' WHEN 3 THEN 'NEU' WHEN 4 THEN 'POS' WHEN 5 THEN 'POS' END AS pr_r_rating, pr_content FROM product_reviews WHERE pmod(pr_review_id, 5) IN (1,2,3)"

In [5]:
df = spark.sql(q).toDF("label", "sentence")

In [28]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(df)

IllegalArgumentException: 'Field "sentence" does not exist.\nAvailable fields: features, FEATURES'

In [27]:
hashingTF = HashingTF(inputCol="words",outputCol="rawFeatures")

NameError: name 'stopwords' is not defined

In [22]:
featurizedData = hashingTF.transform(wordsData)

In [23]:
idf = IDF(inputCol = "rawFeatures",outputCol = "features")

In [24]:
idfModel = idf.fit(featurizedData)

KeyboardInterrupt: 

In [ ]:
rescaledData = idfModel.transform(featurizedData)

In [10]:
df = MLUtils.convertVectorColumnsFromML(colData, "features")

In [11]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["features"], outputCol="FEATURES")

In [12]:
df= vectorAssembler.transform(colData)

In [13]:
parsedData = df.rdd.map(lambda r: r[0])

In [99]:
colData.show()

+--------------------+
|            features|
+--------------------+
|(262144,[12925,50...|
|(262144,[61231,68...|
|(262144,[10300,18...|
|(262144,[11209,70...|
|(262144,[20779,27...|
|(262144,[5987,612...|
|(262144,[25416,25...|
|(262144,[12925,22...|
|(262144,[18375,21...|
|(262144,[9639,183...|
|(262144,[18391,28...|
|(262144,[25416,11...|
|(262144,[6504,104...|
|(262144,[20497,20...|
|(262144,[40140,10...|
|(262144,[329,1292...|
|(262144,[9639,223...|
|(262144,[329,2813...|
|(262144,[21683,37...|
|(262144,[329,1540...|
+--------------------+
only showing top 20 rows



In [14]:
from pyspark.sql import Row
corpus = parsedData.zipWithIndex().map(lambda x: Row (x[0])).toDF()

In [15]:
from pyspark.sql.functions import col
corpus=corpus.select(col("_1").alias("features"))

In [16]:
ldaModel = LDA(k=3, maxIter=10)

In [18]:
model =ldaModel.fit(corpus)

KeyboardInterrupt: 

In [ ]:
topics=model.describeTopics(3)

In [26]:
topics.show(truncate=False)

AttributeError: 'PipelinedRDD' object has no attribute '_jdf'

In [ ]:
ll = model.logLikelihood(corpus)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))

In [ ]:
corpus.printSchema()

In [ ]:
lp = model.logPerplexity(corpus)
print("The upper bound on perplexity: " + str(lp))

In [ ]:
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

In [ ]:
transformed = model.transform(corpus)

In [ ]:
transformed.show(truncate=False)

False

model accuracy 0.87569573283859


TypeError: 'method' object is not iterable

IndentationError: expected an indented block (<ipython-input-80-32f0b963f9ab>, line 3)